## GAN

### Imports

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers, losses, callbacks
from tensorflow.keras.models import save_model, load_model, Model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np
import matplotlib.pyplot as plt
import os
from tensorflow.keras.utils import plot_model, Sequence
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import random as r
from tqdm import tqdm # progress bar
from IPython.display import clear_output
import seaborn as sns
import math

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print(f"GPUs detected: {len(gpus)}")
        print(f"GPUs: {gpus}")
    except RuntimeError as e:
        print(e)
else:
    print("No GPUs detected")

tf.config.optimizer.set_jit(True)  # Active JIT (XLA) globalement

# Set the seed for reproducibility
seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)
r.seed(seed)



### Constants

In [ ]:
class CosineAnnealingScheduler(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, initial_lr, min_lr, cycle_length, cycle_mult=1.0):
        self.initial_lr = initial_lr
        self.min_lr = min_lr
        self.cycle_length = cycle_length
        self.cycle_mult = cycle_mult
        self.current_cycle = 0
        self.iteration = 0

    def __call__(self, step):
        cycle_progress = tf.cast(step % self.cycle_length, tf.float32) / tf.cast(self.cycle_length, tf.float32)
        cosine_decay = 0.5 * (1 + tf.cos(math.pi * cycle_progress))
        lr = self.min_lr + (self.initial_lr - self.min_lr) * cosine_decay
        return lr


# Constants
IMAGE_SHAPE = (64, 64, 3)
BATCH_SIZE = 4*32
EPOCHS = 300
G_RATIO, D_RATIO = 3, 1 # Ratio de mise à jour du générateur et du discriminateur
initial_lr = 2e-4
min_lr = 5e-6
cycle_length = 2000  # nombre de steps avant de redémarrer

gen_lr_schedule = CosineAnnealingScheduler(initial_lr, min_lr, cycle_length)
disc_lr_schedule = CosineAnnealingScheduler(initial_lr, min_lr, cycle_length)


GENERATOR_OPTIMIZER = tf.keras.optimizers.Adam(learning_rate=gen_lr_schedule, beta_1=0.5)
DISCRIMINATOR_OPTIMIZER = tf.keras.optimizers.Adam(learning_rate=disc_lr_schedule, beta_1=0.5)

IDS = [ f'datasets/Humans_Face/{element}' for element in os.listdir('datasets/Humans_Face') ]
IDS += [ f'datasets/celeba/{element}' for element in os.listdir('datasets/celeba') ]
r.shuffle(IDS)
PATIENCE = 2

def inference():
    pass

def plot_inference(image=None):
    result = inference(for_plot=True, image=image)
    fig, axes = plt.subplots(1, len(result), figsize=(20, 5))
    for i, res in enumerate(result):
        step, img = res
        img = (img + 1) / 2 # Convertion de -1, 1 à 0, 1
        axes[i].imshow(np.clip(img, 0, 1))
        axes[i].set_title(f'{int(step)}')
        axes[i].axis('off')
    plt.show()

def array_stats(array, plot=False):
    print(f'Shape: {array.shape} \n \
        Mean: {np.mean(array)} \n \
        Min: {np.min(array)} \n \
        Max: {np.max(array)} \n \
        Std: {np.std(array)}')
    if plot:
        sns.histplot(array.flatten())
        plt.show()


### Dataset

In [ ]:
# Face Dataset
class DatasetGeneratorFace(Sequence):
    def __init__(self, ensemble, **kwargs):
        super().__init__(**kwargs)
        self.ensemble = ensemble
        
        # Créer une liste de tous les IDs d'images
        self.ids = IDS
    
    def __len__(self):
        return int(np.ceil(len(self.ids) / BATCH_SIZE))
    
    def __getitem__(self, index):
        batch_ids = self.ids[index * BATCH_SIZE : (index + 1) * BATCH_SIZE]
        batch_images = []
        for id in batch_ids:
            # Charger l'image
            image = load_img(id, target_size=(IMAGE_SHAPE[0], IMAGE_SHAPE[1]), color_mode='rgb')
            image = img_to_array(image)
            image = image / 255.0 # Normalisation entre 0 et 1
            batch_images.append(image)
        
        batch_images = np.array(batch_images)

        return (batch_images) # On renvoie l'image d'entrée et la cible (auto-encodeur)

TrainGen = DatasetGeneratorFace('train')
print(f"Train: {len(TrainGen)} \n")

# Visualisation des données
def plot_data_gen(gen, n=5):
    fig, axes = plt.subplots(1, n, figsize=(20, 5))
    for i in range(n):
        image = gen[i][0]
        axes[i].imshow(image)
        axes[i].axis('off')
    plt.show()

plot_data_gen(TrainGen, n=5)

    

### Model

In [ ]:
def conv_block(x, filters, kernel_size=3, strides=1, padding='same', activation='leaky_relu', batch_norm=True, transpose=False, dropout=0.0):
    if transpose:
        x = layers.Conv2DTranspose(filters, kernel_size=kernel_size, strides=strides, padding=padding)(x)
    else:
        x = layers.Conv2D(filters, kernel_size=kernel_size, strides=strides, padding=padding)(x)
    if batch_norm:
        x = layers.BatchNormalization()(x)
    if activation:
        x = layers.Activation(activation)(x)
    if dropout > 0.0:
        x = layers.Dropout(dropout)(x)
    return x


def build_GeneratorModel():
    noise_dims = (64,)
    noise_input = layers.Input(shape=(noise_dims))

    x = layers.Reshape((4, 4, 4))(noise_input)

    x = conv_block(x, 256, transpose=True)
    x = conv_block(x, 256, transpose=True)

    x = layers.UpSampling2D(size=(2, 2), interpolation='bilinear')(x)
    x = conv_block(x, 128, transpose=True)
    x = conv_block(x, 128, transpose=True)

    x = layers.UpSampling2D(size=(2, 2), interpolation='bilinear')(x)
    x = conv_block(x, 64, transpose=True)
    x = conv_block(x, 64, transpose=True)

    x = layers.UpSampling2D(size=(2, 2), interpolation='bilinear')(x)
    x = conv_block(x, 64, transpose=True)

    x = layers.UpSampling2D(size=(2, 2), interpolation='bilinear')(x)
    x = conv_block(x, 32, transpose=True)
    x = conv_block(x, 32, transpose=True)

    x = layers.Conv2D(3, kernel_size=3, padding='same')(x)
    x = layers.Activation('sigmoid')(x)

    return Model(inputs=noise_input, outputs=x, name="Generator")

def build_DiscriminatorModel():
    image_input = layers.Input(shape=(64, 64, 3))

    x = conv_block(image_input, 32, dropout=0.05)

    x = conv_block(x, 64, strides=2, dropout=0.05)

    x = conv_block(x, 128, strides=2, dropout=0.05)

    x = conv_block(x, 256, strides=2, dropout=0.05)
    
    x = conv_block(x, 512, strides=2, dropout=0.05)

    x = layers.Flatten()(x)
    x = layers.Dropout(0.4)(x)
    x = layers.Dense(1, activation='sigmoid')(x)

    return Model(inputs=image_input, outputs=x, name="Discriminator")

GeneratorModel, DiscriminatorModel = build_GeneratorModel(), build_DiscriminatorModel()
NOISE_DIM = list(GeneratorModel.input_shape[1:])
print(GeneratorModel.summary())
print(DiscriminatorModel.summary())

### Training

In [ ]:
loss_fn = tf.keras.losses.BinaryCrossentropy(from_logits=False)  # car output= sigmoid

@tf.function(jit_compile=True)
def train_step(real_images, DiscriminatorModel, GeneratorModel, turn):
    batch_size = real_images.shape[0]
    disc_loss, gen_loss = None, None
    # === Génération d'images fausses ===
    noise = tf.random.normal([batch_size] + NOISE_DIM)
    fake_images = GeneratorModel(noise, training=True)

    # === Discriminateur ===
    real_labels = tf.ones((batch_size, 1))
    fake_labels = tf.zeros((batch_size, 1))
    if 'discriminator' in turn :
        with tf.GradientTape() as disc_tape:
            real_output = DiscriminatorModel(real_images, training=True)
            fake_output = DiscriminatorModel(fake_images, training=True)

            disc_loss_real = loss_fn(real_labels, real_output)
            disc_loss_fake = loss_fn(fake_labels, fake_output)
            disc_loss = disc_loss_real + disc_loss_fake

        grads_disc = disc_tape.gradient(disc_loss, DiscriminatorModel.trainable_variables)
        DISCRIMINATOR_OPTIMIZER.apply_gradients(zip(grads_disc, DiscriminatorModel.trainable_variables))
    # === Générateur ===
    if 'generator' in turn:
        noise = tf.random.normal([batch_size] + NOISE_DIM)
        with tf.GradientTape() as gen_tape:
            generated_images = GeneratorModel(noise, training=True)
            fake_output = DiscriminatorModel(generated_images, training=True)
            gen_loss = loss_fn(real_labels, fake_output)  # veut tromper le disc

        grads_gen = gen_tape.gradient(gen_loss, GeneratorModel.trainable_variables)
        GENERATOR_OPTIMIZER.apply_gradients(zip(grads_gen, GeneratorModel.trainable_variables))
    
    return gen_loss, disc_loss

best_val_loss = float("inf")
wait = 0
for epoch in range(EPOCHS):
    print(f"\nEpoch {epoch+1}/{EPOCHS}")
    # --- Entraînement ---
    progress_bar = tqdm(TrainGen, desc="Training", leave=False)
    for index, real_images in enumerate(progress_bar):
        turn= ['discriminator'] if index % (G_RATIO + D_RATIO) < D_RATIO else ['generator']
        if index == 0:
            turn = ['discriminator', 'generator']
        g_loss, d_loss = train_step(real_images, DiscriminatorModel, GeneratorModel, turn)
        current_lr = GENERATOR_OPTIMIZER._decayed_lr(tf.float32).numpy()
        if g_loss is not None:
            pb_g_loss = g_loss
        if d_loss is not None:
            pb_d_loss = d_loss
        progress_bar.set_postfix({"g_loss": f"{pb_g_loss:.4e}", "d_loss": f"{pb_d_loss:.4e}", "lr": f"{current_lr:.3e}"})

        # --- Test ---
        if index % 100 == 0:
            noise = tf.random.normal([5] + NOISE_DIM)
            fake_images = GeneratorModel(noise, training=False)
            fig, axes = plt.subplots(1, len(fake_images), figsize=(20, 5))
            for i, img in enumerate(fake_images):
                axes[i].imshow(img)
                axes[i].axis('off')
            plt.show()

    # --- Sauvegarde du modèle ---
    if epoch % 5 == 0:
        save_model(GeneratorModel, f"models/gan/GeneratorModel.h5")
        save_model(DiscriminatorModel, f"models/gan/DiscriminatorModel.h5")
        print(f"Models saved at epoch {epoch}")
